In [10]:
import sys
sys.path.append("../src")  

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from modelo_autoregresivo import ModeloAutoregresivo
from generacion import GeneradorTexto 
import pickle
import json
import math

In [27]:
# Cargar datos
with open("secuencias.pkl", "rb") as f:
    secuencias = pickle.load(f)

In [28]:
# Preparamos datos para entrenamiento
x_train = []
y_train = []

for seq in secuencias:
    if len(seq) < 2:
        continue
    x_train.append(seq[:-1])  # input sin el último token
    y_train.append(seq[1:])   # target sin el primero

# Padding manual para igualar longitudes
max_len = max(len(seq) for seq in x_train)

for i in range(len(x_train)):
    x_train[i] += [0] * (max_len - len(x_train[i]))
    y_train[i] += [0] * (max_len - len(y_train[i]))

x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

In [29]:
# Función de pérdida y optimizador
criterio = nn.CrossEntropyLoss(ignore_index=0)
optimizador = torch.optim.Adam(modelo.parameters(), lr=0.001)

# Entrenamiento simple
modelo.train()
epocas = 20

for epoca in range(epocas):
    total_loss = 0
    for i in range(len(x_train)):
        entrada = x_train[i].unsqueeze(0)
        objetivo = y_train[i].unsqueeze(0)

        logits = modelo(entrada)
        logits = logits.view(-1, vocab_size)
        objetivo = objetivo.view(-1)

        loss = criterio(logits, objetivo)

        optimizador.zero_grad()
        loss.backward()
        optimizador.step()

        total_loss += loss.item()

    perplejidad = math.exp(total_loss / len(x_train))
    print(f"Época {epoca+1}, Pérdida promedio: {total_loss / len(x_train):.4f}, Perplejidad: {perplejidad:.2f}")

# Guardar el modelo entrenado
torch.save(modelo.state_dict(), "modelo_entrenado.pth")


Época 1, Pérdida promedio: 0.2504, Perplejidad: 1.28
Época 2, Pérdida promedio: 0.3676, Perplejidad: 1.44
Época 3, Pérdida promedio: 0.2243, Perplejidad: 1.25
Época 4, Pérdida promedio: 0.1473, Perplejidad: 1.16
Época 5, Pérdida promedio: 0.1699, Perplejidad: 1.19
Época 6, Pérdida promedio: 0.2219, Perplejidad: 1.25
Época 7, Pérdida promedio: 0.1865, Perplejidad: 1.20
Época 8, Pérdida promedio: 0.2041, Perplejidad: 1.23
Época 9, Pérdida promedio: 0.1773, Perplejidad: 1.19
Época 10, Pérdida promedio: 0.2153, Perplejidad: 1.24
Época 11, Pérdida promedio: 0.2836, Perplejidad: 1.33
Época 12, Pérdida promedio: 0.1545, Perplejidad: 1.17
Época 13, Pérdida promedio: 0.1398, Perplejidad: 1.15
Época 14, Pérdida promedio: 0.1649, Perplejidad: 1.18
Época 15, Pérdida promedio: 0.1356, Perplejidad: 1.15
Época 16, Pérdida promedio: 0.1655, Perplejidad: 1.18
Época 17, Pérdida promedio: 0.1582, Perplejidad: 1.17
Época 18, Pérdida promedio: 0.0972, Perplejidad: 1.10
Época 19, Pérdida promedio: 0.1687, P

In [30]:
# Cargar vocabulario 
with open("vocab.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)
inv_vocab = {v: k for k, v in vocab.items()}
vocab_size = len(vocab)

In [31]:
# Crear modelo
modelo = ModeloAutoregresivo(vocab_size)
modelo.load_state_dict(torch.load("modelo_entrenado.pth"))
modelo.eval()

ModeloAutoregresivo(
  (embedding): Embedding(23, 128)
  (pos_encoder): PositionalEncoding()
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0

In [32]:
# Crear generador
generador = GeneradorTexto(modelo, inv_vocab)

secuencia_inicial = "ho"
entrada = torch.tensor([[vocab[c] for c in secuencia_inicial]])

# Generar texto
texto_generado = generador.generar_greedy(entrada, num_tokens=30)
print("Texto generado:", texto_generado)


Texto generado: hod uemu unoueueueueueueueueueue
